In [200]:
# Store runs predict path
out_path = "/workspace/viettel-ai-challenge-track-ds/runs/private-test-attempt-2"
import os 
os.makedirs(out_path, exist_ok=True)

In [201]:
# Change file here
train_file = "/workspace/viettel-ai-challenge-track-ds/data/training_set.csv"
public_test_file = "/workspace/viettel-ai-challenge-track-ds/data/public_testset.csv"
private_test_file = "/workspace/viettel-ai-challenge-track-ds/data/test_set_private.csv"

In [202]:
!nvidia-smi

Tue Nov 19 21:30:50 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.239.06   Driver Version: 470.239.06   CUDA Version: 12.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:01:00.0 Off |                  Off |
| 75%   93C    P2   135W / 140W |  15252MiB / 16117MiB |     92%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A4000    Off  | 00000000:21:00.0 Off |                  Off |
| 64%   

In [258]:
# Choose GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# SELFRec Models

In [204]:
%cd /workspace/viettel-ai-challenge-track-ds/SELFRec/

/workspace/viettel-ai-challenge-track-ds/SELFRec


/opt/conda/envs/rapids-24.10/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [205]:
from util.conf import ModelConf
from data.loader import FileIO
import torch
import torch.nn as nn
import torch.nn.functional as F
from base.graph_recommender import GraphRecommender
# from util.conf import OptionConf
from util.sampler import next_batch_pairwise
from base.torch_interface import TorchGraphInterface
from util.loss_torch import bpr_loss, l2_reg_loss, InfoNCE

In [206]:
import pandas as pd
import numpy as np

df = pd.read_csv(train_file)
df = df[["UserId", "ItemId", "Purchase"]]
df = df.groupby("UserId").tail(20)

user_list = df.groupby('UserId')['ItemId'].nunique()
# user_list_denoise = user_list[user_list<=30].index.to_list()
user_list_denoise = user_list.index.to_list()

In [207]:
test = pd.read_csv(private_test_file, names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
test_user_id = test['user_id'].values

In [208]:
final_user_list = []
final_user_list.extend(user_list_denoise)
final_user_list.extend(test_user_id)
user_list_denoise = list(set(final_user_list))

In [209]:
import pandas as pd
import numpy as np

df = pd.read_csv(train_file)
predict_df = pd.read_csv(private_test_file, names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])

test_user_id = predict_df['user_id'].values

item_columns = predict_df.columns[1:]  # Lấy tất cả cột trừ cột uid
item_in_test_df = pd.unique(predict_df[item_columns].values.ravel())
#Lọc bỏ user_id bị lẫn trong tập test
user_list = df.UserId.unique()
item_in_test_df = list(set(item_in_test_df).difference(set(user_list)))

test_df = df[~df.ItemId.isin(item_in_test_df)].sample(frac=0.05, random_state=42)  # 90% for train
# train_df = df
train_df = df.drop(test_df.index)  # Remaining 10% for test

train_df.shape, test_df.shape

((375372, 4), (14551, 4))

In [210]:
# Save train and test sets as .txt files without headers, separated by a space
train_df.to_csv("train.txt", index=False, header=False, sep=" ")
test_df.to_csv("test.txt", index=False, header=False, sep=" ")

## Import Code Model 

In [211]:
from data.loader import FileIO
class SELFRec(object):
    def __init__(self, config):
        self.social_data = []
        self.feature_data = []
        self.config = config
        self.training_data = FileIO.load_data_set(config['training.set'], config['model']['type'])
        self.test_data = FileIO.load_data_set(config['test.set'], config['model']['type'])

        self.kwargs = {}
#         if config.contain('social.data'):
#             social_data = FileIO.load_social_data(self.config['social.data'])
#             self.kwargs['social.data'] = social_data
        # if config.contains('feature.data'):
        #     self.social_data = FileIO.loadFeature(config,self.config['feature.data'])
        print('Reading data and preprocessing...')

    def execute(self):
        # import the model module
#         import_str = f"from model.{self.config['model']['type']}.{self.config['model']['name']} import {self.config['model']['name']}"
#         exec(import_str)
        recommender = f"{self.config['model']['name']}(self.config,self.training_data,self.test_data,**self.kwargs)"
        return eval(recommender)

In [212]:
from model.graph.LightGCN import *
from model.graph.XSimGCL import *
from model.graph.DirectAU import *
from model.graph.SimGCL import *

## Get result

In [213]:
predict_df = pd.read_csv(private_test_file, names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
predict_df.head()

,user_id,item_id_1,item_id_2,item_id_3,item_id_4,item_id_5,item_id_6,item_id_7,item_id_8,item_id_9,...,item_id_991,item_id_992,item_id_993,item_id_994,item_id_995,item_id_996,item_id_997,item_id_998,item_id_999,item_id_1000
0,0a9Fr9I8Gr,boqO5yZLGn,8ZkMmPx7LA,lY9LS9tm2o,GlcugvT0Yc,CgxImEqsPz,Gob6GYLouL,IRvMMJ0b7w,40N3veG6jC,ZDcoul5cdR,...,Z8xiRnRjZD,5MwdFYt8z0,IzLVUKs7uQ,Ic5BpxKXZg,F9jblvzWuK,xvKZji6pWJ,arvytMaCpS,XC7SvTAElR,oWsTKxTtvA,iLKSmk2T06
1,Jg3UwpR1cA,FR8wGxRAuG,dYzjsJalSw,46UB3fKUvW,43pBc5JjXG,UMBBUX1BCV,QJMMePEj0j,bGCcG8kfTI,1S3CqINx38,iPCMUOS4gu,...,84YaBhvrnw,uUg3iZKEDz,VS9vm70j8u,mb8iada716,W4LaO2AnPF,oH6lYVrH2Z,RdMLs1XP8z,cjhX33aANo,JVR40qZevF,m5tbhqzC7m
2,k0mNOjn7pv,XY3JQdn2uK,VXSaWxoJhS,Jd9W82sCwm,APxSylqBUi,xSYbWMYXQ3,nO5G68rbFr,akZrO7ylK0,u2BPbQVjhn,0mCZtLlSSJ,...,LcT9aU6sDk,erjkmeJrO2,DtAWzeNoqo,9C1LcSgNrd,RfaOiwln47,EjaSqbgfWm,5MmscAd5ND,4CDZjAxiJ3,iPzOhrroRk,Or6J4bnaJo
3,Uw0zXc8Sr8,CZIxh1jK1E,HDkA9U2Xcw,B5Y4WsLlpa,TtcC5Je1JD,4CGmtKJuwQ,Q3UHYGUanE,ZtFOy1wdyW,GDKM1k4EhP,CxB3Ckr0Qg,...,UmYJDzVAm1,6c9wVM3C7n,yy78DDyFwz,JBWS6uOW6t,uFtLv6hcKO,Z8p8xxx0cx,Py42KZCHJP,ewfqfR6fiC,rdZDIeJ1ia,OgmtWQ39sq
4,vLDKNL74C4,geQIsKSQMq,Fv1Qr0KyxC,rOrsptpv8t,x9uKie7B9k,j3BAm5H5Ht,rMoQ4U14Bz,HszEQFR8mO,Vjl5VpE6mQ,Be0DFaA4f2,...,MUWZaAPDNT,rnp6yH6OD5,45KRQI5RJi,iPRCkgoeGJ,6PtfgezcfB,w83nfFLEQF,aD2c2V2wYx,Cu2tQGKjrm,Ec3YhNEIRi,ax59cfAF2Y


In [217]:
model_path = {
    "LightGCN": "/workspace/viettel-ai-challenge-track-ds/runs/lightgcn_final_data/model.pkl",
    # "XSimGCL": "",
    "DirectAU": "/workspace/viettel-ai-challenge-track-ds/runs/directau_final_data/model.pkl",
    "SimGCL": "/workspace/viettel-ai-challenge-track-ds/runs/simgcl_final_data/model.pkl"
}

In [218]:
from util.algorithm import find_k_largest
def test(self):
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        print(f'\rProgress: [{"+" * ratenum}{" " * (50 - ratenum)}]{ratenum * 2}%', end='', flush=True)

    rec_list = {}
    data_train = pd.DataFrame(self.data.training_data, columns= ['uid', 'iid', 'rating'])
    self.data.train_set = data_train[data_train['uid'].isin(test_user_id)].values.tolist()
    user_count = len(self.data.train_set)
    
    for i, user in enumerate(self.data.train_set):
        user = user[0]
        candidates = self.predict(user)
        rated_list, _ = self.data.user_rated(user)
        for item in rated_list:
            candidates[self.data.item[item]] = -10e8
        # ids, scores = find_k_largest(1000, candidates)
        item_names = predict_df[predict_df.user_id == user].values[0][1:]
        scores = []
        for item in item_names:
            try:
                id_tmp = self.data.item[item]
                scores.append(candidates[id_tmp])
            except:
                # Cần sửa khuyến nghị cold start
                scores.append(0)
        
        sorted_list = sorted(list(zip(item_names, scores)), key=lambda x: x[1], reverse=True)
        rec_list[user] = sorted_list
        if i % 1000 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    return rec_list

In [219]:
import pickle
import numba
import gc
import torch
for model, path in model_path.items():
    with open(path, "rb") as f:
        rec = pickle.load(f)

    rec_list = test(rec)

    data = []
    for user_id in test_user_id:
        data.append([user_id] + [i[0] for i in rec_list[user_id]])

    pd.DataFrame(data).to_csv(f'{out_path}/{model}_predict.csv', index = False, header=False)

    del rec
    gc.collect()  # collecting garbage
    torch.cuda.empty_cache()  # cleaning GPU cache

Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%


# RecVAE

In [265]:
%cd /workspace/viettel-ai-challenge-track-ds

/workspace/viettel-ai-challenge-track-ds


/opt/conda/envs/rapids-24.10/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [266]:
vae_checkpoint = "runs/vae_multiverse_chaos"

In [267]:
df = pd.read_csv(f"{vae_checkpoint}/data.csv")

In [268]:
test = pd.read_csv(f"{private_test_file}", header=None)

In [269]:
id2item_df = pd.read_csv(f"{vae_checkpoint}/unique_sid.txt", header=None).rename(columns={0: "ItemId"}).reset_index()

In [270]:
# tail = df.groupby("UserId").tail(50)

In [271]:
test_tr = df[df["UserId"].isin(test[0].values)]
# test_tr = tail[tail["UserId"].isin(test[0].values)]
test_tr = test_tr.merge(id2item_df, how="inner", on="ItemId").rename(columns={"index": "sid"})
user_test = pd.DataFrame(test_tr["UserId"].unique(), columns=["UserId"]).reset_index().rename(columns={"index" : "uid"})
test_tr = pd.merge(test_tr, user_test, on="UserId")
test_tr[["uid", "sid"]].to_csv(f"{vae_checkpoint}/testset_recvae.csv", index=False)

In [272]:
!python recvae/infer.py --dataset $vae_checkpoint --hidden-dim 2048 --latent-dim 1024 --infer_data $vae_checkpoint/testset_recvae.csv --model_path $vae_checkpoint/model.pt 

[0. 1.]
(4225, 82652)


In [273]:
import pickle
with open(f"{vae_checkpoint}/result_csp.pkl", "rb") as f:
    result = pickle.load(f)

id2profile = dict(user_test.values)
profile2id = {value: key for key, value in id2profile.items()}
id2item = dict(id2item_df.values)
item2id = {value: key for key, value in id2item.items()}

In [274]:
import numpy as np
from tqdm.notebook import tqdm
return_list = []
for r in tqdm(test.merge(user_test, left_on=0, right_on="UserId", how="left").values):
    userid = r[0]
    list_item = r[1:1001]
    uid = r[1001]
    
    # Process User not in Test By get the default list
    if np.isnan(uid):
        return_list.append([userid, *list_item])
        continue

    # Convert ItemId to indexs of sparse vector
    item_indexs = []
    for l in list_item:
        try:
            item_indexs.append((item2id[l], l))
        except:
            pass

    # Score and sorted to get to recommend item
    scored_item = []
    indexes, itemids = map(list,zip(*item_indexs))
    for item, score in zip(itemids, result[int(uid)][indexes]):
        scored_item.append((item, score))

    scored_item = sorted(scored_item, key=lambda x : x[1], reverse=True)
    recommend_list, _ = map(list,zip(*scored_item))

    # Append to return list to make submit
    return_list.append([userid, *recommend_list])

  0%|          | 0/4225 [00:00<?, ?it/s]

In [275]:
return_df = pd.DataFrame(return_list)

In [276]:
return_df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0a9Fr9I8Gr,EALxnioXfL,PRanViElTN,6pPEL58mZO,xRrrBsbzL7,jZu5JNmd3k,sLoTA2aPEl,HvjPW7BXnD,ZXR0UOYMC5,Cpoe5fD1ft,...,None,None,None,None,None,None,None,None,None,None
1,Jg3UwpR1cA,Pyze1t4kzV,dYzjsJalSw,mECjdAi7IP,RVU66qOwYq,DyrmcAo1bZ,BJWIvAhZ5R,MtHzVtuxq5,4SqbvdwOGp,5g0f4SiT73,...,2rqp06ENF3,TYtJNgdTQT,LB4tn3Pu0R,kvxgqEihmT,jRTea0c0IA,None,None,None,None,None
2,k0mNOjn7pv,jAE0cnq4rG,y5jQg8Y9Jx,vDGrqJgbwa,wbKXQYFoKH,sBqcTvsj1B,FVFqnj3to2,mdabgpB5Xc,BEUea91tmp,nqeHWxod5x,...,lU3SwNH391,XEBugry5iK,rL2HJmNdlO,MFTqPsfgEv,YDE0Oh45lt,None,None,None,None,None
3,Uw0zXc8Sr8,51ANpugrpL,Daytgu5kmV,tMe0DRMMyG,S41d9ELmhC,9H1pnwbhUi,7f7LkOKsAr,0LNX4ggWMG,4QqMKOUjwH,ZodOCtOkxr,...,None,None,None,None,None,None,None,None,None,None
4,vLDKNL74C4,PMsUraEglo,uzts0Vtcry,gQCGJIJOaM,LBoipMRS7v,3btZEflaP0,nOcItm8Qkk,BPL6wy2Aig,WyogFojRKA,dmMV4izrqE,...,2fB6qZNgFp,nyh8MgNceK,Kw9YRCkD7o,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4220,SIFzQe6gnn,l6DuJC6Rg2,8f9Vtw6DqP,49OyD4VQvZ,0krwZa3Mav,XmphVhomUp,0ZnXQFRpOt,4Pm47rVPdy,RJwil82Eri,32FfxVfh3i,...,None,None,None,None,None,None,None,None,None,None
4221,fa6Y1KVdoh,SUfmHWyfXf,Cqqm8eQ1If,lBocTTcnRM,J5KwfCDaaf,Jl8JMSuXVM,VWAFPk5W8O,a6Gsd0EzuU,owC4Cs0nZD,5M9RUEfrbb,...,leDHP1bybl,JfbZl5Hh4s,U9esFzogVx,Gct8qKZaEz,BeLbx6QVEs,JoT6E3RB1K,None,None,None,None
4222,xO9WRD2Zbm,qi8qLmgATK,v9ELdECPMi,dch1WQaq9W,v4yt3Xk1RV,X2wn0fgqwH,ooHfH960JX,VQrdlRqIcc,Ibp0sHM5hA,MWRRsijMN6,...,yhSmJMOgf6,oWH4Dd5X7N,NbD4ouH6Rl,None,None,None,None,None,None,None
4223,EW403CecBe,kGYz42FXiw,gUQQbWny7I,RBLWPNcXTK,0tv68hDDTQ,3Mc5uFofhd,JopKOis5wb,jXVPOKYrBZ,OwVO8kJZ85,BTslUx1sTE,...,rlprN2eiuY,5de2lizMFg,xRUJJpZD7j,qdBPlaIUq2,dzUMySPGz9,lO9EWKFZ96,T5rJlVwG5H,None,None,None


In [263]:
# Fill remanining cells as nan value to make submit file eligible
for i in range(len(return_df.columns), 1001):
    return_df[i] = np.nan

In [264]:
return_df.fillna("0").to_csv(f"{out_path}/RecVAE_madness.csv", header=None, index=False)

# ALS

In [195]:
als_checkpoint = "/workspace/viettel-ai-challenge-track-ds/runs/ALS_final_data/"

In [198]:
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k
from implicit.evaluation import ndcg_at_k

In [ ]:
from implicit.gpu import matrix_factorization_base

In [ ]:
def submit(model, csr_train, test_user_id, item_names, submission_name="predict_BPR.csv"):
    preds = []
    batch_size = 2000
    # Make sure we're only predicting for users in test_user_id
    to_generate = np.array([user_id for user_id in test_user_index])  # Make sure users exist in user_ids
    
    for startidx in range(0, len(to_generate), batch_size):
        batch = to_generate[startidx:startidx + batch_size]
        # print(batch)
        ids, scores = model.recommend(batch, csr_train[batch], N=1000, filter_already_liked_items=True)
        
        for i, userid in enumerate(batch):
            customer_id = user_ids[userid]
            user_items = ids[i]
            
            # Filter the items to keep only those in item_names for the current user
            valid_item_ids = [item for item in user_items if item_ids[item] in item_names[customer_id]]
            
            # If fewer than 1000 items are valid, fill the rest with invalid items or random items
            # You can adjust the fill logic if needed, here it's just taking invalid items or the first items.
            invalid_items = [item for item in user_items if item_ids[item] not in item_names[customer_id]]
            filled_items = valid_item_ids + invalid_items[:(1000 - len(valid_item_ids))]
            
            # Ensure we have exactly 1000 items
            article_ids = [item_ids[item_id] for item_id in filled_items[:1000]]
            
            preds.append([customer_id] + article_ids)
    
    # Create the DataFrame for submission
    df_preds = pd.DataFrame(preds, columns=['customer_id'] + [f'item_{i}' for i in range(1000)])
    df_preds.to_csv(submission_name, index=False, header=False)
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds


In [ ]:
item_names = {}

for user_id in test_user_id:
    item_names[user_id] = test[test.user_id == user_id].values[0][1:] 

In [ ]:
%%time

df_preds = submit(model, csr_train, test_user_id, item_names);

# Ensemble